# Refactor of baseline model

In this notebook the code defined in the [notebook 03_TrainBaselineModel.ipynb](../1_build_end2end_prototype/03_TrainBaselineModel.ipynb) is refactored into functions. These functions are placed in the script xxxxx to easily run W&B sweeps for hyperparameter tuning.

In [1]:
import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback
import torchvision.models as tvmodels

import params
# Helper functions - for example metrics we will track during our experiments
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

In [2]:
# SimpleNamespace - creates an object to store values as attributes without creating your own (almost empty) class.

train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    arch="resnet18"
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
    log_preds=True
)

In [ ]:
# Initialise wandb run?????????????????????????????????????

def download_data():
    # Use artefacts to track the data linage of our models
    processed_data_artifact = wandb.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
    # Download split data from W&B artifact
    processed_dataset_dir = Path(processed_data_artifact.download())
    return processed_dataset_dir

In [ ]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

In [ ]:
def get_df(processed_dataset_dir, is_test=False):
    # Read csv containing data split data (train/valid/test)
    df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

    if not is_test:
        # Remove test set rows
        df = df[df.Stage != 'test'].reset_index(drop=True)
        # is_valid column will tell our trainer how we want to split data between training and validation.
        df['is_valid'] = df.Stage == 'valid'
    else:
        df = df[df.Stage == 'test'].reset_index(drop=True)

    # Add image and mask label paths to dataframe
    df["image_fname"] = [processed_dataset_dir/f'images/{f}' for f in df.File_Name.values]
    df["label_fname"] = [label_func(f) for f in df.image_fname.values]

    return df

In [ ]:
# fastAI dataloader
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

In [ ]:
def log_predictions(learn):
    "Log a Table with model predictions"
    samples, outputs, predictions = get_predictions(learn)
    table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
    wandb.log({"pred_table":table})

In [ ]:
def log_final_metrics(learn):
    scores = learn.validate()
    metric_names = ['final_loss'] + [f'final_{x.name}' for x in learn.metrics]
    final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
    for k,v in final_results.items(): 
        wandb.summary[k] = v

In [ ]:
def train(config):
    # Set seed for reproducibility.
    set_seed(train_config.seed, reproducible=True)

    # - pass train_config into W&B run to control training hyperparameters
    # - project=params.WANDB_PROJECT to make this W&B run be part of same project as previous  notebook W&B runs
    run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=config)

    config = wandb.config

    processed_dataset_dir = download_data()
    df = get_df(processed_dataset_dir)

    dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

    # We use intersection over union metrics: mean across all classes (MIOU) and IOU for each class separately.
    metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
            TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

    # The model is a unet based on a pretrained resnet18 backbone
    # - backbone type obtained from torchvision.modles (defined in config variable)
    learn = unet_learner(dls, arch=getattr(tvmodels, config.arch), pretrained=config.pretrained, metrics=metrics)

    callbacks = [
        SaveModelCallback(monitor='miou'),              # Save model with best miou metric
        WandbCallback(log_preds=False, log_model=True)  # We log predictions manually on W&B (so set log_preds=False), and we want to log model W&B (so log_model=True)
    ]

    # Train model
    learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)

    if config.log_preds:
        log_predictions(learn)

    log_final_metrics(learn)

    wandb.finish()

# Run training

In [ ]:
train(train_config)